# Set up Environment

In [1]:
import gymnasium as gym
import random

In [8]:
%pip install atari

ERROR: Could not find a version that satisfies the requirement atari (from versions: none)
ERROR: No matching distribution found for atari
Note: you may need to restart the kernel to use updated packages.


In [11]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

/Users/rish/opt/anaconda3/lib/python3.9/site-packages/gymnasium/envs/registration.py:578: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [12]:
actions # Two directions of movement 

2

## Loading different Envirnoment and finding actions

In [4]:
env = gym.make("ALE/Adventure-v5")
states = env.observation_space.shape[0]
actions = env.ction_space.n

NamespaceNotFound: Namespace ALE not found. Have you installed the proper package for ALE?

## Continuing on

In [ ]:
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1])
        observation, reward, terminated, truncated, info = env.step(action)
        
    print(episode, score)

/Users/rish/opt/anaconda3/lib/python3.9/site-packages/gymnasium/envs/classic_control/cartpole.py:212: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v0", render_mode="rgb_array")
  gym.logger.warn(
/Users/rish/opt/anaconda3/lib/python3.9/site-packages/gymnasium/envs/classic_control/cartpole.py:177: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


In [ ]:
# I am going to continue in  ajupyter notebook in the anaconda app